# Set working directory

In [1]:
import os
cwd = os.path.split(os.getcwd())
if cwd[-1] == 'tutorials':
    os.chdir('..')
# TODO assert in the correct directory
!python --version

Python 3.8.2


# Import modules

In [2]:
from BRON.build_BRON import build_graph
from path_search.path_search_BRON import main_attack
from utils.bron_network_utils import load_graph_nodes
from utils.tutorial_util import print_files_in_folder

# Build BRON

The different types of threat data are linked together in a graph called BRON. The data types are linked with bidirectional edges in the following manner:
```
Tactic <--> Technique <--> CAPEC <--> CWE <--> CVE <--> CPE
```
To create a new BRON using input threat data, run the following command:
```
python BRON/build_BRON.py --input_data_folder INPUT_DATA_FOLDER --save_path SAVE_PATH --only_recent_cves (optional)
``` 
`INPUT_DATA_FOLDER` is the folder path to input threat data, and `SAVE_PATH` is the folder path to save the BRON graph and files. The `SAVE_PATH` folder should contain subfolders organized as follows: `SAVE_PATH/BRON/original_id_to_bron_id`. To make BRON using only recent CVEs, add the argument `--only_recent_cves`. An example BRON with its input threat data can be found in the `example_data` folder.

In [3]:
save_path = 'example_data/example_output_data'
input_data_folder = 'example_data/example_input_data'
build_graph(save_path, input_data_folder)

In [4]:
print_files_in_folder(save_path)

'example_data/example_output_data/BRON 1598530105.965684'
'example_data/example_output_data/all_cves_all_versions 1598530105.9690936'
'example_data/example_output_data/all_cves_latest_version 1598530105.9699273'
('example_data/example_output_data/search_result_ibm_business_process_manager.csv '
 '1598530105.9832911')
'example_data/example_output_data/figures 1601316149.9301994'
'example_data/example_output_data/search_result_tactic.csv 1601389591.769105'
'example_data/example_output_data/search_result_technique.csv 1601389591.869807'
'example_data/example_output_data/BRON.json 1601396716.0232089'


# Structure of BRON

Each entry of threat data is a node in BRON that has 4 attributes. The node has a unique name in BRON of the form (threat data type)\_(unique 5 digit id) where the threat data type is either Tactic, Technique, CAPEC, CWE, CVE, or CPE.

There are 4 attributes for each node:
- Original_id: ID of threat data in MITRE/NIST if it exists
- Datatype: One of Tactic, Technique, CAPEC, CWE, CVE, or CPE
- Name: Name of threat data in MITRE/NIST if it exists
- Metadata: Any additional information that is contained in MITRE/NIST 

# Find paths in BRON

Once BRON is built, it can be searched for paths that contain certain data types or specific nodes. The path search starts with a set of starting points and then finds all paths in BRON from those starting points. A severity score based on the CVSS score is calculated for each path that is found. To find paths in BRON, run the following command:
```
python -m path_search.path_search_BRON --BRON_folder_path BRON_FOLDER_PATH --starting_point STARTING_POINT --starting_point_type STARTING_POINT_TYPE --results_file RESULTS_FILE --length (optional)
```
`BRON_FOLDER_PATH` is the folder path to the BRON graph that is being searched, `STARTING_POINT` is the file path to the CSV file that contains the starting point IDs, `STARTING_POINT_TYPE` is the data type of the starting points (either Tactic, Technique, CAPEC, CWE, CVE, or CPE), and `RESULTS_FILE` is the file path to store the CSV results file.

To make the results contain only the _number_ of nodes in a path instead of a set of node names, add the argument `--length`.

The output of path search is a CSV file that shows how the different data types are related to each other. The paths start at the type of argument and then search the graph to find the connections to other data. For example, if the input file consisted of 3 CAPECs: 100, 105, 200, then we would see all of the Tactics, Techniques, CWEs, CVEs, and CPEs associated with each CAPEC individually. The final row contains the total count of each data type in BRON. An example starting point file and its path search result can be found in the `example_data` folder.

In [5]:
starting_point_type = 'tactic'
starting_point = os.path.join(input_data_folder, f'starting_point_{starting_point_type}.csv')
results_file = os.path.join(save_path, f'search_result_{starting_point_type}.csv')
length = False
main_attack(BRON_folder_path=save_path,
            args_file=starting_point,
            results_name=results_file,
            arg_type=starting_point_type,
            length=length)
# TODO change risk_score to cvss_score (a better name)
# TODO print also the actual (not only bron ID)

,tactic,technique,capec,cwe,cve,risk_score,cpe,attack,Node Type
0,{tactic_00008},"{technique_00077, technique_00175, technique_0...","{capec_00308, capec_00301, capec_00311, capec_...","{cwe_00347, cwe_00519, cwe_00546, cwe_00522, c...","{cve_01536, cve_01045, cve_01459, cve_01167}",25.70,"{cpe_01044, cpe_01168, cpe_01533, cpe_01458}",NaN,connected
1,{tactic_00014},"{technique_00027, technique_00098, technique_0...","{capec_00325, capec_00283, capec_00339, capec_...","{cwe_00914, cwe_00412, cwe_00502, cwe_00346}","{cve_01448, cve_01449, cve_01047, cve_01144, c...",101.50,"{cpe_01444, cpe_01101, cpe_01251, cpe_01249, c...",NaN,connected
2,{tactic_00012},"{technique_00016, technique_00127, technique_0...","{capec_00308, capec_00301, capec_00289, capec_...","{cwe_00357, cwe_00420, cwe_00422, cwe_00653, c...","{cve_01518, cve_01496, cve_01169, cve_01254, c...",101.95,"{cpe_01407, cpe_01410, cpe_01391, cpe_01286, c...",NaN,connected
3,12,266,519,251,217,1014.00,277,0.0,Total Number


In [6]:
starting_point_type = 'technique'
starting_point = os.path.join(input_data_folder, f'starting_point_{starting_point_type}.csv')
results_file = os.path.join(save_path, f'search_result_{starting_point_type}.csv')
length = False
main_attack(BRON_folder_path=save_path,
            args_file=starting_point,
            results_name=results_file,
            arg_type=starting_point_type,
            length=length)

,tactic,technique,capec,cwe,cve,risk_score,cpe,attack,Node Type
0,{tactic_00004},{technique_00003},{},{},{},0.0,{},NaN,connected
1,12,266,519,251,217,1014.0,277,0.0,Total Number
